In [137]:
import osmnx as ox
import networkx as nx
from scipy.sparse import csr_matrix
import scipy.sparse as sp
import pandas as pd
import pickle

from collections import namedtuple

import dgl
from dgl.nn import GraphConv
import torch
from torch import nn
import torch.nn.functional as F

### Data Extraction

In [12]:
data = pd.read_pickle("data/training/Graph_State_Uni.pkl")

In [16]:
data = data.dropna()
data.head()

,Experiment,Town,County,State,Treatment,Search_Place,Graph
0,"(AlAMU, 1875)",Huntsville,Madison,Alabama,1,"Huntsville, Alabama","(56922621, 56923776, 56923781, 56923787, 56924..."
1,"(AlAMU, 1875)",Montgomery,Montgomery,Alabama,0,"Montgomery, Alabama","(58878326, 58878327, 58878330, 58878370, 58878..."
3,"(AubU, 1856)",Auburn,Lee,Alabama,1,"Auburn, Alabama","(56844738, 56844741, 56845079, 56845082, 56845..."
4,"(AubU, 1856)",Florence,Lauderdale,Alabama,0,"Florence, Alabama","(56540270, 56540280, 56540305, 56540703, 56540..."
5,"(AubU, 1856)",Talladega,Talladega,Alabama,0,"Talladega, Alabama","(52315445, 52318540, 52326293, 52387532, 52398..."


In [133]:
def get_nodes(graphs):
    
    all_features_list = []
    
    for G in graphs:
        nodes = list(G.nodes)
        degree_tensor = torch.tensor(list(G.degree()))
        in_degree_tensor = torch.tensor(list(G.in_degree()))
        all_features_tensor = torch.cat([degree_tensor.unsqueeze(1), in_degree_tensor.unsqueeze(1)], dim=1)
        all_features_list.append(all_features_tensor)

    combined_feature_tensor = torch.cat(all_features_list, dim=0)
    return combined_feature_tensor

In [141]:
def get_edges(graphs):
    
    all_sparse_tensors = []
    
    for G in graphs:
        adj_matrix = nx.to_scipy_sparse_array(G, dtype=float, format='csr')
        sparse_tensor = torch.sparse_coo_tensor(adj_matrix.tocoo())
        all_sparse_tensors.append(sparse_tensor)
        
    return all_sparse_tensors

In [134]:
node_features = get_nodes(data["Graph"])

In [142]:
edge_features = get_edges(data["Graph"])

TypeError: sparse_coo_tensor(): argument 'size' (position 1) must be tuple of ints, but found element of type coo_array at pos 0

### Data Preprocessing

In [56]:
class AttentionPool(nn.Module):
      """ 
      
      Attention-based pooling layer for graph classification.
      Handles graphs of different sizes. 
      
      """

      def __init__(self, units):
        super(AttentionPool, self).__init__()
        self.units = units
        self.attention_dense = nn.Linear(units, units)
        self.score_dense = nn.Linear(units, 1)

      def forward(self, node_features, adj_matrix):
        # Calculate attention scores
        attention_inputs = self.attention_dense(node_features)
        attention_logits = torch.matmul(attention_inputs, attention_inputs.t())
        # Mask out self-attention
        attention_logits -= torch.eye(attention_logits.size(0)) * 1e9
        attention_weights = F.sigmoid(self.score_dense(attention_logits))

        # Apply attention to node features
        pooled_features = torch.matmul(attention_weights, node_features)

        return pooled_features

In [61]:
def pool_features(node_features, edge_matrices):
    pooled_features_list = []
    for node_features_batch, adj_matrix_batch in zip(node_features, edge_matrices):
        attention_pool = AttentionPool(units=128) # adjust?
        pooled_features = attention_pool(node_features_batch, adj_matrix_batch)
        pooled_features_list.append(pooled_features)
    return pooled_features_list

In [62]:
test_node = node_features[0:5]
test_edge = edge_features[0:5]
test_pool = pool_features(test_node, test_edge)

TypeError: linear(): argument 'input' (position 1) must be Tensor, not list

### Model

In [7]:
class GCNConv(nn.Module):
     """Graph Convolutional Layer"""
       
     def __init__(self, in_features, out_features):
       super(GCNConv, self).__init__()
       self.weight = nn.Linear(in_features, out_features)

     def forward(self, node_features, adj_matrix):
       # Normalize adjacency matrix
       adj_matrix = F.normalize(adj_matrix, dim=1, p=1, eps=1e-9)
       # Perform graph convolution
       x = torch.matmul(adj_matrix, torch.matmul(node_features, self.weight))
       return x

In [8]:
class GraphCNN(nn.Module):
      """Graphical CNN model for network classification."""
    
      def __init__(self, in_features, hidden_features):
        super(GraphCNN, self).__init__()
        self.gcn1 = GCNConv(in_features, hidden_features)
        self.gcn2 = GCNConv(hidden_features, hidden_features)
        self.attention_pool = AttentionPool(hidden_features)
        self.fc1 = nn.Linear(hidden_features, 64)
        self.fc2 = nn.Linear(64, 2)
    
      def forward(self, node_features, adj_matrix):
        x = self.gcn1(node_features, adj_matrix)
        x = F.relu(x)
        x = self.gcn2(x, adj_matrix)
        x = F.relu(x)
        x = self.attention_pool(x, adj_matrix)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)
        output = F.sigmoid(self.fc2(x))
        return output
    